# 자기상관(autocorrelation)
- 작성자: 고려대학교 경제학과 한치록 교수, 데이터사이언스팀 이창훈 과장</br></br>
시계열 자료는 시간에 걸쳐서 자기상관을 갖는 것이 특징 중의 하나이다. 오차항에 자기상관이 있더라도, 설명변수가 strictly exogenous하면(즉, 모든 시기의 설명변수가 모든 시기의 오차항과 비상관이면) OLS 추정량은 여전히 비편향(unbiased)이다. 하지만 오차항에 자기상관이 있으면 OLS에 대하여 $s^2 (X'X)^{-1}$에 기초하여 통상적으로 계산하는 표준오차는 타당성을 잃고 이에 근거한 t검정이나 F검정은 타당하지 않다. OLS 추정량에 대하여 표준오차를 구하려면 Newey-West 추정량 등 HAC 표준오차를 사용해야 한다. 이에 대해서는 [Robust Inferences with OLS][3_02]에서 살펴보았다. 이제는 오차항 자기상관과 관련된 나머지 주제들을 살펴본다. 관련 주제는 다음과 같다.

1. 오차항 자기상관 검정
2. Cochrane-Orcutt 추정
3. Prais-Winsten 추정

설명변수는 여전히 strictly exogenous하다고 우선 가정한다. 회귀모형이 $y_t = X_t\beta + u_t$이고 $X_t$가 strictly exogenous라는 것은 $E(u_t|X_1, \ldots, X_T)=0$ for all $t$임을 의미한다. 즉, $X_t$는 과거 정보로부터 예측할 대상인 것이 아니라, 전체 기간에 대하여 그 값이 주어진 것으로 간주한다. 참고로 $y_{t-1}$은 predetermined (또는 weakly exogenous)이나 strictly exogenous하지 않다. 표본크기가 큰 경우 $E(X_t'u_t)=0$으로 가정을 완화할 수 있는데 이 경우 OLS 추정량은 biased이나 consistent하다.

[3_02]: 02%20Robust%20Inferences%20with%20OLS.ipynb

## 오차항 자기상관 검정

### Durbin-Watson 검정

설명변수가 strictly exogenous인 $y_t = X_t\beta + u_t$ 모형에서 $u_t$에 시계열상관이 있는지, 즉 서로 다른 $t$와 $s$에서 $u_t$와 $u_s$가 상관되는지를 검정하는 문제를 고려하자. 가장 유명한 검정은 [Durbin-Watson 검정][DW]이다. Residual이 $\hat{u}_t$라 할 때, Durbin-Watson 통계량은 다음과 같다.
$$d = \sum_{t=2}^T (\hat{u}_t - \hat{u}_{t-1})^2 / \sum_{t=1}^T \hat{u}_t^2$$
파이썬에서 이 값은 linear model을 fit한 것의 `summary()`에 포함되어 있다. 직접 계산하려면 다음과 같이 한다.
```python
statsmodels.stats.stattools.durbin_watson(resids, axis=0)
```
문제는 DW 검정의 $p$값을 구하는 것이다. 과거에는 주어진 표본크기와 설명변수 개수에 따라 임계값의 상한선과 하한선을 구하여 비교하는 방법을 사용했으나, 이제는 [Farebrother (1980)](https://academic.oup.com/jrsssc/article-abstract/29/2/224/6954000?redirectedFrom=fulltext)가 제시한 알고리즘(표본크기가 작을 때는 Pan 알고리즘, 클 때에는 Imhof의 근사값)을 사용하여 $p$값을 계산한다. [R][R]의 `lmtest` 패키지에 `dwtest` 명령으로 이 알고리즘이 구현되어 있다. 파이썬 Statsmodels는 이 알고리즘을 구현하지 않으며, 별도의 `dwtest` 모듈(2023. 12. 30. 현재 버전 0.0.4)이 이를 구현한 듯하다.
```sh
pip install dwtest
```

[DW]: (https://en.wikipedia.org/wiki/Durbin–Watson_statistic)
[R]: https://r-project.org/

In [1]:
import pandas as pd
import numpy as np

In [3]:
from dwtest import dwtest

df = pd.read_stata('../data/klein.dta') # stata의 dta 확장자 불러오기
dwtest("consump ~ wagegovt", data = df)

(np.float64(0.3217998163194888), np.float64(2.564162873834575e-06))

위 결과에서 검정통계량 값은 [R]의 계산 결과와 일치하나, $p$값은 약간 다르다.
```r
# R codes
DF <- readstata13::read.dta13('klein.dta')
dwtest(consump~wagegovt, data=DF)

#     Durbin-Watson test
#
# data:  consump ~ wagegovt
# DW = 0.3218, p-value = 0.000000006478
# alternative hypothesis: true autocorrelation is greater than 0
```

$p$값이 원체 작아서 차이가 있는 것으로 보이는지, 아니면 R와 Python 둘 중 하나의 `dwtest`에서 $p$값을 구하는 부분에 문제가 있는 것인지 확인하기 위해, 귀무가설하에서 데이터를 생성하여(그러면 $p$값이 더 클 것이라 기대하고) 파이썬 `dwtest` 패키지와 [R]의 결과를 비교해 보자.

[R]: https://r-project.org/

In [4]:
# Generate data
np.random.seed(1)
df = pd.DataFrame({'x': np.random.normal(size=50)})
df['y'] = 1-df['x'] + np.random.normal(size=50)

# Write data to file
df.to_csv('../data/dwtest.csv', index = False)

# Read data
df = pd.read_csv('../data/dwtest.csv')
dwtest('y~x', data=df)

(np.float64(1.5871075583157497), np.float64(0.07225768432275743))

위 데이터를 이용하여 R의 `dwtest` 명령을 사용하면 결과는 다음과 같다.
```r
# R codes
dwtest(y~x, data=read.csv('dwtest.csv'))

#     Durbin-Watson test
# 
# data:  y ~ x
# DW = 1.5871, p-value = 0.07226
# alternative hypothesis: true autocorrelation is greater than 0
```

파이썬 `dwtest`와 결과가 같다. 파이썬 `dwtest`를 안심하고 사용해도 좋을 듯하다.

### Breusch-Godfrey 검정

잔차에 자기상관이 있으면 OLS 회귀계수는 여전히 consistent하지만 분산 추정이 틀려져 표준오차가 부정확해지고, 그 결과 가설검정(t-검정, F-검정)에서 잘못된 결론(제1종 오류/제2종 오류)을 내릴 수 있다. Breusch (1979)와 Godfrey (1978)은 OLS 잔차 $\hat{u}_t$을 원래의 설명변수들과 $\hat{u}_{t-1}, \ldots, \hat{u}_{t-q}$에 대하여 OLS 회귀할 것을 제안하였다(한치록, 계량경제학강의 제4판). 이 회귀의 $R^2$값을 이용해 잔차의 시차항이 통계적으로 유의미한가를 검정한다. 귀무가설($H_0$)은 "잔차에 자기상관이 없다"이며, 대립가설($H_1$)은 "어느 하나 이상의 시차잔차가 유의하다" 이다. Breusch-Godfrey 검정은 파이썬 Statsmodels에 [statsmodels.stats.diagnostic.acorr_breusch_godfrey]((https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_breusch_godfrey.html))로 구현되어 있다(). 최대 래그 수($q$)를 `nlags=`로 지정할 수 있으며, 디폴트로 사용되는 값은 $[n/5]$와 10 중 더 작은 값이다.

In [5]:
import statsmodels.formula.api as smf
import statsmodels.stats.diagnostic as smsd

df = pd.read_csv('../data/dwtest.csv')
ols = smf.ols('y~x', data=df).fit()
smsd.acorr_breusch_godfrey(ols, nlags=3)

(np.float64(7.524839014736245),
 np.float64(0.05692365451821859),
 2.657378632660237,
 0.05966712653653038)

위 데이터셋에서 $n=50$이므로 디폴트로 사용되는 래그 수는 10인데 너무 커서 3으로 지정하였다. 결과의 첫 번째 줄은 LM 통계량 값, 두 번째 줄은 LM 통계량의 $p$값, 세 번째 줄은 F 통계량 값, 네 번째 줄은 F 통계량의 $p$값이다. R이 구한 결과는 다음과 같다.
```r
# R codes
bgtest(y~x, data=read.csv('dwtest.csv'), order=3)

#     Breusch-Godfrey test for serial correlation of order up to 3
#
# data:  y ~ x
# LM test = 7.5248, df = 3, p-value = 0.05692
```

LM 부분의 결과가 똑같은 것을 확인할 수 있다.

## 추정법

### Cochrane-Orcutt 추정

회귀모형이 $y_t = \alpha + X_t\beta + u_t$이고 오차항 $u_t$가 $u_t = \rho u_{t-1} + e_t$, $e_t\sim iid\; (0, \sigma_e^2)$이라 하자. $y_t$에서 $\rho y_{t-1}$을 빼면 다음이 된다.
$$y_t - \rho y_{t-1} = (1-\rho) \alpha + (X_t - \rho X_{t-1}) \beta + e_t, \;\; t=2,\ldots,n$$
이 회귀식의 오차항 $e_t$에는 자기상관이 없다. 만약 $\rho$가 알려져 있다면 $y_t - \rho y_{t-1}$을 상수항($1-\rho$)과 $X_t-\rho X_{t-1}$에 대하여 $t=2,\ldots, n$의 표본을 이용하여 OLS를 하면 $\beta$를 추정할 수 있다. 이 추정법을 Cochrane-Orcutt 추정법이라 한다. $\rho$를 알지 못하는 경우에는 $\rho=0 \to \beta \to \rho \to \beta$의 2개 단계를 거치거나 수렴할 때까지 반복한다. [Stata]에서는 `prais` 명령에 `corc` 옵션을 사용한다.
```stata
/* Stata */
prais y x1 x2, corc twostep
prais y x1 x2, corc
```

주어진 $\beta$에서 $\rho$를 추정하는 방법으로 Stata의 `prais`는 다음 6가지를 구현한다([Stata prais 매뉴얼](https://www.stata.com/manuals/tsprais.pdf) 참조).

* `regress`: $\hat{u}_t$를 $\hat{u}_{t-1}$에 절편 없이 회귀할 때의 기울기 추정치 사용 (default)
* `freg`: $\hat{u}_t$를 $\hat{u}_{t+1}$에 절편 없이 회귀할 때의 기울기 추정치 사용(forward regression)
* `tscorr`: $\sum_{t=2}^n \hat{u}_t \hat{u}_{t-1} / \sum_{t=1}^T \hat{u}_t^2$ 사용
* `dw`: $1-\text{dw}/2$ 사용
* `theil`: $\rho_{\mathrm{tscorr}} (n-k)/n$ 사용
* `nagar`: $(\rho_{\mathrm{dw}}\cdot n^2 + k^2) / (n^2 - k^2)$ 사용

2단계 추정은 다음과 같이 될 것이다.

1. OLS 추정을 하여 잔차 $\hat{u}_t$를 구한다.
2. [Stata]의 `regress` 옵션에 해당하는 방법을 사용한다면 $\hat{u}_t$를 $\hat{u}_{t-1}$에 OLS 회귀를 하여 $\hat\rho$을 구한다. 다른 방식을 사용한다면 다른 방식으로 $\hat\rho$을 구한다.
3. 종속변수를 $y_t - \hat\rho y_{t-1}$로 바꾸고 독립변수들을 $X_t - \hat\rho X_{t-1}$로 바꾸어서 OLS 회귀를 한다($t=2,\ldots,n$ 사용). 상수항도 1이 아니라 $1-\hat\rho$으로 변환됨에 유의하라.

반복된 추정은 다시 1번으로 돌아가는데 OLS 추정이 아니라 위 3번에서 구한 $\hat\beta$를 이용하여 잔차를 $y_t - X_t\hat\beta$로 계산한다. 이 절차를 반복하다가 직전 단계 $\hat\rho$과 이번 단계 $\hat\rho$이 거의 똑같으면 멈춘다. '거의 똑같은' 것의 기준은 사용자가 정하며 [Stata]는 기본값으로 `1e-6`을 사용한다.

[Stata]: https://stata.com/

### Prais-Winsten FGLS

Cochrane-Orcutt 추정법은 $t=2,\ldots,n$을 사용하므로 표본크기가 1개 줄어든 셈이다. [Prais-Winsten 추정법][prais]은 Cochrane-Orcutt 추정에서 사용되는 관측치들에 추가하여 $t=1$에 해당하는 관측치를 추가로 사용하는데 적절히 변환하여 $u_t$가 AR(1)이라는 가정하에서 이분산이 없도록 만들어 준다. 구체적으로, 오차항이 covariance stationary한 AR(1)이라고 가정하면, $var(u_1) = \sigma_e^2 /(1-\rho^2)$이고 $var(u_t - \rho u_{t-1}) = \sigma_e^2$이므로 관측치들을 다음과 같이 변환한다.
$$\begin{split}
\sqrt{1-\rho^2} y_1 &= \sqrt{1-\rho^2} \alpha + \sqrt{1-\rho^2} X_1\beta + \sqrt{1-\rho^2} u_1,\\
y_t - \rho y_{t-1} &= (1-\rho) \alpha + (X_t - \rho X_{t-1}) \beta + e_t, \;\; t=2,\ldots, n
\end{split}$$
이렇게 변환한 오차항들은 오차항 $u_t$가 AR(1)이고 covariance stationary일 때 모두 동일한 분산을 갖고 서로간에 공분산이 0이다(white noise). 위 변환된 식에 OLS를 적용하는 것이 [Prais-Winsten 추정법][prais]이다. Stata에서는 `prais` 명령을 사용한다.
```stata
/* Stata */
prais y x1 x2, twostep
prais y x1 x2
```
주어진 $\beta$에서 $\rho$를 추정하는 방법들 및 몇 단계를 거칠 것인지는 위의 Cochrane-Orcutt 추정 소절에서 설명한 바와 같다.

2단계 추정은 앞의 Cochrane-Orcutt 추정에서 3단계만 바꾼다.

1. OLS 추정을 하여 잔차 $\hat{u}_t$를 구한다.
2. [Stata]의 `regress` 옵션에 해당하는 방법을 사용한다면 $\hat{u}_t$를 $\hat{u}_{t-1}$에 OLS 회귀를 하여 $\hat\rho$을 구한다. 다른 방식을 사용한다면 다른 방식으로 $\hat\rho$을 구한다.
3. $t=2,\ldots,n$에서는 종속변수를 $y_t - \hat\rho y_{t-1}$, 독립변수들을 $X_t - \hat\rho X_{t-1}$로 바꾸고(상수항도 1이 아니라 $1-\hat\rho$으로 변환됨에 유의하라), $t=1$에서는 $y_1$을 $\sqrt{1-\hat\rho^2} y_1$로, $X_1$을 $\sqrt{1-\hat\rho^2} X_1$으로 변환하여(상수항도 1에서 $\sqrt{1-\hat\rho^2}$으로 변환됨에 유의) OLS 회귀를 한다.

반복된 추정은 다시 1번으로 돌아가는데 OLS로부터의 잔차가 아니라 위 3번 단계에서 구한 $\hat\beta$을 사용한 잔차 $y_t - X_t\hat\beta$을 사용한다는 점은 앞의 Cochrane-Orcutt 추정의 경우와 같다.

[Stata]: https://stata.com/
[prais]: https://en.wikipedia.org/wiki/Prais–Winsten_estimation

### Prais 모듈

`tol =  1e-6` 범위 내에 들어올 때까지 반복하는 추정의 예는 다음과 같다. Stata 매뉴얼의 예제를 복원하고자 한다. 코드의 정확성을 확인하기 위하여 [Stata] 실행 결과를 살펴보면 다음과 같다.

```stata
. webuse qsales, clear

. prais csales isales, corc two

Iteration 0:   rho = 0.0000
Iteration 1:   rho = 0.6312

Cochrane–Orcutt AR(1) regression with twostep estimates

      Source |       SS           df       MS      Number of obs   =        19
-------------+----------------------------------   F(1, 17)        =   3453.63
       Model |  15.5749186         1  15.5749186   Prob > F        =    0.0000
    Residual |  .076665287        17  .004509723   R-squared       =    0.9951
-------------+----------------------------------   Adj R-squared   =    0.9948
       Total |  15.6515839        18  .869532438   Root MSE        =    .06715

------------------------------------------------------------------------------
      csales | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
      isales |   .1737583   .0029567    58.77   0.000     .1675202    .1799964
       _cons |  -1.068526   .4533976    -2.36   0.031    -2.025112   -.1119411
-------------+----------------------------------------------------------------
         rho |   .6311623
------------------------------------------------------------------------------
Durbin–Watson statistic (original)    = 0.734728
Durbin–Watson statistic (transformed) = 1.650247
```

위는 two step Cochrane-Orcutt 추정이다. 다음은 수렴 때까지 반복하는 Prais-Winsten 추정이다.

```stata
. prais csales isales

Iteration 0:   rho = 0.0000
Iteration 1:   rho = 0.6312
Iteration 2:   rho = 0.6500
Iteration 3:   rho = 0.6528
Iteration 4:   rho = 0.6532
Iteration 5:   rho = 0.6533
Iteration 6:   rho = 0.6533
Iteration 7:   rho = 0.6533
Iteration 8:   rho = 0.6533

Prais–Winsten AR(1) regression with iterated estimates

      Source |       SS           df       MS      Number of obs   =        20
-------------+----------------------------------   F(1, 18)        =  13848.94
       Model |  60.8198448         1  60.8198448   Prob > F        =    0.0000
    Residual |  .079049867        18  .004391659   R-squared       =    0.9987
-------------+----------------------------------   Adj R-squared   =    0.9986
       Total |  60.8988947        19  3.20520498   Root MSE        =    .06627

------------------------------------------------------------------------------
      csales | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
      isales |   .1749874   .0023848    73.38   0.000     .1699773    .1799976
       _cons |  -1.267818   .3549307    -3.57   0.002      -2.0135   -.5221362
-------------+----------------------------------------------------------------
         rho |   .6532947
------------------------------------------------------------------------------
Durbin–Watson statistic (original)    = 0.734728
Durbin–Watson statistic (transformed) = 1.711080
```

위와 동일한 결과를 주는 파이썬 모듈이 본 프로젝트에서 개선된 `statsmodels` 패키지에 `statsmodels.regression.linear_model.Prais`로 제공되어 있다. 원래 `statsmodels` 패키지에는 포함되어 있지 않다. Twostep Cochrane-Orcutt 추정 결과는 다음과 같다. 최초에 import bok_da를 해야 statsmodels에서 Prais를 쓸 수 있다는 점에 주의해야 한다.

[Stata]: https://stata.com/

In [10]:
import bok_da
import statsmodels.api as sm
from statsmodels.regression.linear_model import Prais

qsales = pd.read_stata('../data/qsales.dta') # stata의 dta 확장자 불러오기
mod = sm.Prais.from_formula('csales~isales', data = qsales, corc=True, twostep=True)
co = mod.iterative_fit()
print(co.summary(slim=True))

Iteration 0:  rho = 0.0000
Iteration 1:  rho = 0.6312
       Cochrane-Orcutt AR(1) regression with twostep estimates Results        
Dep. Variable:                 csales   R-squared:                      0.9951
Model:                          Prais   Adj. R-squared:                 0.9948
No. Observations:                  19   F-statistic:                     3454.
Covariance Type:            nonrobust   Prob (F-statistic):             0.0000
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -1.06853    0.45340      -2.36      0.031    -2.02511    -0.11194
isales        0.17376    0.00296      58.77      0.000     0.16752     0.18000
rho                                    0.631162
Durbin-Watson statistic (original)     0.734728
Durbin-Watson statistic (transformed)  1.650247
* convergence not achieved
--------------------------

Notes:
1. Standard Errors assume tha

`twostep`, `corc` 등의 인자를 주고 실험해 볼 수 있다. 수렴을 하는 경우에는 Stata와 결과가 일치한다.

*Note*. [Statsmodels][sm]의 [GLSAR](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.GLSAR.html) 모델이 AR 모형 추정을 해 주는 것으로 보이는데, 이를 사용해서는 위 결과와 일치하는 결과를 얻지 못하였다.

[sm]: https://www.statsmodels.org/

다음으로 반복적인 Prais-Winsten 추정 결과는 다음과 같다. 결과는 앞의 Stata 결과와 동일하다.

In [11]:
from bok_da.linear.lm import prais

pw = prais('csales~isales', qsales)
print(pw.summary(slim=True))

Iteration 0:  rho = 0.0000
Iteration 1:  rho = 0.6312
Iteration 2:  rho = 0.6500
Iteration 3:  rho = 0.6528
Iteration 4:  rho = 0.6532
Iteration 5:  rho = 0.6533
Iteration 6:  rho = 0.6533
Iteration 7:  rho = 0.6533
Iteration 8:  rho = 0.6533
       Prais-Winsten AR(1) regression with iterative estimates Results        
Dep. Variable:                 csales   R-squared:                      0.9987
Model:                          Prais   Adj. R-squared:                 0.9986
No. Observations:                  20   F-statistic:                 1.385e+04
Covariance Type:            nonrobust   Prob (F-statistic):             0.0000
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -1.26782    0.35493      -3.57      0.002    -2.01350    -0.52214
isales        0.17499    0.00238      73.38      0.000     0.16998     0.18000
rho                                    0.65329